In [1]:

import pandas as pd
import numpy as np

#upload the file /Users/tommi/Desktop/Uni_documents/Data/commodity-2005.csv taking the "Dates" column as index
commodity = pd.read_csv('/Users/tommi/Desktop/Uni_documents/Data/commodity-2005.csv', index_col='Dates')

In [2]:
# import /Users/tommi/Desktop/Uni_documents/Data/volatility-2004.csv
volatility = pd.read_csv('/Users/tommi/Desktop/Uni_documents/Data/volatility-2004.csv', index_col='Dates')


In [3]:
#upload /Users/tommi/Desktop/Uni_documents/Data/EHI640_returns10-02-2025.csv
benchmark_vol = pd.read_csv('/Users/tommi/Desktop/Uni_documents/Data/EHI640_returns10-02-2025.csv')


In [4]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
#from the volatility dataframe, delete the rows where data is less than 1/12/2004 and greater than 31/12/2004
volatility = volatility.loc['31/12/2004': '31/12/2024' ]

#reset the index of volatility dataframe
volatility.reset_index(inplace=True)

#merge the benchmark_vol and volatility dataframes, the first line with the first , the second with the second and so on 
merged = pd.merge(benchmark_vol, volatility, left_index=True, right_index=True)

#create a new df with the merged data for when the Dates are greater then 31/01/2011
df = merged[merged["Dates"] > "31/01/2011"]

# Convert percentage returns to decimal form
df.loc[:, "PercentReturn"] = pd.to_numeric(df["PercentReturn"], errors="coerce") / 100

# List of proxy columns
proxy_cols = ["VIXY US Equity", "VXX US Equity", "UVXY US Equity", "VIXM US Equity", "VIX Index"]

# Compute log returns for proxies
for proxy in proxy_cols:
    df.loc[:, proxy + "_log_return"] = np.log(df[proxy] / df[proxy].shift(1))

# Drop NaNs resulting from shift operation
df.dropna(subset=["PercentReturn"] + [p + "_log_return" for p in proxy_cols], inplace=True)

# Dictionary to store R² values
r2_scores = {}

# Run regressions for each proxy
for proxy in proxy_cols:
    proxy_return_col = proxy + "_log_return"
    temp_df = df.dropna(subset=[proxy_return_col])  # Remove NaNs for this proxy
    if len(temp_df) > 10:  # Ensure enough data points
        X = sm.add_constant(temp_df[proxy_return_col])  # Add intercept
        y = temp_df["PercentReturn"]
        model = sm.OLS(y, X).fit()
        r2_scores[proxy] = model.rsquared  # Store R²

# Convert R² results into a DataFrame for comparison
r2_df = pd.DataFrame(list(r2_scores.items()), columns=["Proxy", "R²"]).sort_values(by="R²", ascending=False)

# Highlight the best proxy
best_proxy = r2_df.iloc[0]["Proxy"]
print("Best Proxy by R²:", best_proxy)
print(r2_df)


Best Proxy by R²: VIXM US Equity
            Proxy        R²
3  VIXM US Equity  0.611010
1   VXX US Equity  0.610285
0  VIXY US Equity  0.576338
2  UVXY US Equity  0.560874
4       VIX Index  0.419153


/var/folders/ly/mr0404x501n5fsqtt6yhhbtm0000gn/T/ipykernel_5216/3429752284.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, proxy + "_log_return"] = np.log(df[proxy] / df[proxy].shift(1))
/var/folders/ly/mr0404x501n5fsqtt6yhhbtm0000gn/T/ipykernel_5216/3429752284.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, proxy + "_log_return"] = np.log(df[proxy] / df[proxy].shift(1))
/var/folders/ly/mr0404x501n5fsqtt6yhhbtm0000gn/T/ipykernel_5216/3429752284.py:24: SettingWithCopyWarning: 
A

In [15]:
#commodity head
commodity.head()

,BCOM Index,PDBC US Equity,DBC US Equity,GSG US Equity,USCI US Equity,DJP US Equity,SPY US Equity,UC14 LN Equity,COMF LN Equity
Dates,,,,,,,,,
04/01/05,142.3930,NaN,NaN,NaN,NaN,NaN,118.83,NaN,NaN
05/01/05,142.1799,NaN,NaN,NaN,NaN,NaN,118.01,NaN,NaN
06/01/05,144.4881,NaN,NaN,NaN,NaN,NaN,118.61,NaN,NaN
07/01/05,144.2124,NaN,NaN,NaN,NaN,NaN,118.44,NaN,NaN
10/01/05,144.5722,NaN,NaN,NaN,NaN,NaN,119.00,NaN,NaN


In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

df = commodity

# List of proxy columns
proxy_cols = [
    "PDBC US Equity", "DBC US Equity", "GSG US Equity", 
    "USCI US Equity", "DJP US Equity", "SPY US Equity", 
    "UC14 LN Equity", "COMF LN Equity"
]

# Convert index and proxies to log returns
df["BCOM_log_return"] = np.log(df["BCOM Index"] / df["BCOM Index"].shift(1))

for proxy in proxy_cols:
    df[proxy + "_log_return"] = np.log(df[proxy] / df[proxy].shift(1))

# Drop NaNs resulting from shift operation
df = df.dropna(subset=["BCOM_log_return"] + [p + "_log_return" for p in proxy_cols])

# Dictionary to store R² values
r2_scores = {}

# Run regressions for each proxy
for proxy in proxy_cols:
    proxy_return_col = proxy + "_log_return"
    temp_df = df.dropna(subset=[proxy_return_col])  # Remove NaNs for this proxy
    if len(temp_df) > 10:  # Ensure enough data points
        X = sm.add_constant(temp_df[proxy_return_col])  # Add intercept
        y = temp_df["BCOM_log_return"]
        model = sm.OLS(y, X).fit()
        r2_scores[proxy] = model.rsquared  # Store R²

# Convert R² results into a DataFrame for comparison
r2_df = pd.DataFrame(list(r2_scores.items()), columns=["Proxy", "R²"]).sort_values(by="R²", ascending=False)

# Highlight the best proxy
best_proxy = r2_df.iloc[0]["Proxy"]
print("Best Proxy by R²:", best_proxy)
print(r2_df)




Best Proxy by R²: DJP US Equity
            Proxy        R²
4   DJP US Equity  0.921244
1   DBC US Equity  0.768072
2   GSG US Equity  0.723998
7  COMF LN Equity  0.661192
3  USCI US Equity  0.649206
6  UC14 LN Equity  0.610042
0  PDBC US Equity  0.502229
5   SPY US Equity  0.082434
